| 컬럼명 | 설명 | 값 의미 | 비고 |
|:---:|:---|:---|:---|
| **Churn** | 고객 이탈 여부 | **1**: 이탈 (Yes), **0**: 유지 (No) |
| **Age** | 고객의 나이 | 정수형 (int) | |
| **Region** | 거주 지역 | 범주형 (str) | Seoul, Metro, Other |
| **Tenure_Months** | 서비스 가입 기간 | 정수형 (int) | 개월 수 단위 |
| **Total_Usage_GB** | 최근 데이터 사용량 | 실수형 (float) | GB 단위 |
| **Support_Tickets_3M**| 최근 3개월 CS 문의 건수 | 정수형 (int) | 고객 불만 지표 |
| **NPS_Score** | 고객 만족도 점수 | 정수형 (int) | -100 ~ 100 점 |
| **Monthly_Fee** | 월 납부 요금 | 실수형 (float) | |
| **Late_Payments_6M** | 최근 6개월 연체 횟수 | 정수형 (int) | |
| **Contract** | 계약 형태 | 범주형 (str) | Month-to-month, 1-year, 2-year |
| **AutoPay** | 자동결제 여부 | 이진형 (bool) | 1: 사용, 0: 미사용 |
| **Internet_Type** | 회선 종류 | 범주형 (str) | Fiber, DSL, 5G |
| **Has_Addon** | 부가서비스(0/1)| 정수형(int) | |

# 1. 데이터 전처리

In [ ]:
import pandas as pd

df = pd.read_csv('../../data/churn_train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
import koreanize_matplotlib # noqa: F401
import matplotlib.pyplot as plt

t = df['Churn'].value_counts()
labels = ['유지', '이탈']

fig, ax = plt.subplots()
ax.pie(t, labels=labels, autopct='%1.1f%%')


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

encoder = OrdinalEncoder()
scaler = StandardScaler()

col = ["Contract", "Internet_Type", "Region"]
df[col] = encoder.fit_transform(df[col].copy())

df['Monthly_Fee'] = scaler.fit_transform(df[['Monthly_Fee']])

corr_matrix = df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu', fmt='.2f', linewidths=0.5)
plt.title('Feature Correlation with Churn')
plt.show()

In [ ]:
import seaborn as sns

cols = ['Tenure_Months', 'Support_Tickets_3M', 'Late_Payments_6M', 'Contract', 'AutoPay', 'NPS_Score', 'Churn']

sns.pairplot(data=df[cols], hue='Churn', plot_kws={'alpha': 0.4})

In [ ]:
col = ['Tenure_Months', 'Total_Usage_GB', 'Support_Tickets_3M', 'Late_Payments_6M', 'AutoPay', 'NPS_Score']
n = len(col)

fig, ax = plt.subplots(n, 2, figsize=(12, 5 * n))

# plt.figure(figsize=(15, 10))
for i, col in enumerate(col):
    # (왼쪽) KDE Plot: 분포의 모양 비교
    sns.kdeplot(data=df, x=col, hue='Churn', fill=True, ax=ax[i, 0])
    plt.title(f'{col}에 따른 이탈 분포 (KDE)')
    
    # (오른쪽) Box Plot: 중앙값과 이상치 비교
    sns.boxplot(data=df, x='Churn', y=col, ax=ax[i, 1])
    plt.title(f'이탈 여부별 {col} 차이 (Box)')

plt.show()

# 2. 모델 학습

In [ ]:
import pandas as pd

test_df = pd.read_csv('../../data/churn_test.csv')
test_df.head()

In [ ]:
df.head()

In [ ]:
test_df.info()

In [ ]:
X_train = df.drop(['Customer_ID', 'Churn'], axis=1)
y_train = df['Churn']

In [ ]:
print(X_train.shape)


# 3. 평가

## 1) LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from lib.vizkit import print_evaluation

col = ["Contract", "Internet_Type", "Region"]
fee_col = ["Monthly_Fee"]

X_test = test_df.drop(['Customer_ID', 'Churn'], axis=1)
y_test = test_df['Churn']

X_test[col] = encoder.transform(test_df[col])

X_test[fee_col] = scaler.transform(test_df[fee_col])

model = LogisticRegression(max_iter=5000, class_weight='balanced', C=10.0)
model.fit(X_train, y_train)

pred = model.predict(X_test)

# 3. ROC-AUC (분류 모델의 종합 점수)
# 확률값이 필요하므로 predict_proba를 사용합니다.
pred_proba = model.predict_proba(X_test)[:, 1]

print_evaluation(y_test, pred, pred_proba)

In [ ]:
# 오차 핼렬
from lib.vizkit import show_confusion_matrix

show_confusion_matrix(y_test, pred)

## 2) RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from lib.vizkit import print_evaluation

rf_model = RandomForestClassifier(
    n_estimators=300,               # 나무 개수를 늘림
    max_depth=10,                   # 너무 깊으면 과적합되니 10 정도로 제한
    min_samples_leaf=5,             # 한 잎사귀에 최소 5명은 있게 해서 일반화
    class_weight='balanced',        # 불균형 해소 유지
    random_state=3333
)

rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_proba = rf_model.predict_proba(X_test)[:, 1]

print_evaluation(y_test, rf_pred, rf_proba)


In [ ]:
# 오차 핼렬
from lib.vizkit import show_confusion_matrix

show_confusion_matrix(y_test, rf_pred)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=importances)
plt.title('Random Forest Feature Importance')
plt.show()

In [ ]:
import xgboost as xgb
from lib.vizkit import print_evaluation

xgb_model = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth=3, eval_metric="logloss")

xgb_model.fit(X_train, y_train)

xgb_pred = xgb_model.predict(X_test)
xgb_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print_evaluation(y_test, pred, xgb_pred_proba)


In [ ]:
from lib.vizkit import show_confusion_matrix

show_confusion_matrix(y_test, xgb_pred)